In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time
from pathlib import Path
import pandas as pd
import re

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
path_to_circuits = '../../circuits/qasm/'
input_data = {"Name": [], "circuit": [], "graph": []}
dataframes = []

gates = []
t_count = []
cliffords = []
cnot = []
other = []
hadamard = []
times = []

for file in Path(path_to_circuits).glob('*.qasm'):
    circuit = zx.Circuit.load(file).to_basic_gates()
    # if circuit.qubits <= 19 and circuit.qubits >= 8 and len(circuit.gates) <= 5000 and len(circuit.gates) >= 100:
    if file.stem == "gf2^5_mult" or file.stem == "gf2^6_mult" or file.stem == "barenco_tof_3" or file.stem == "mod_red_21":

        if not file.stem == "ham15-med":
            try:
                circuit = zx.optimize.basic_optimization(circuit)
            except Exception as e:
                pass
            graph = circuit.to_graph()
            graph = graph.copy()

            input_data["Name"].append(file.stem)
            input_data["circuit"].append(circuit)
            input_data["graph"].append(circuit.to_graph())
            logger.info(f"Loaded {file.stem}")
            logging.info(circuit.stats())

            numbers = re.findall(r'\d+', circuit.stats())

            # Assign the numbers to variables
            gates.append(int(numbers[1]))
            t_count.append(int(numbers[2]))
            cliffords.append(int(numbers[3]))
            cnot.append(int(numbers[6]))
            other.append(int(numbers[7]))
            hadamard.append(int(numbers[8]))
            times.append(0)

# Define the column names
columns = input_data["Name"]

# Define the row labels
rows = ["Gates", "T-Count", "Cliffords", "CNOTS", "Other 2 Qubit Gates", "Hadamard", "Time"]

#Define the algorithm
algorithm = ["OR", "TR", "FR", "G0", "G1", "G2", "G3", "GN"]
# algorithm = ["OR", "TR", "FR", "G0", "G1", "G2", "G3", "GN", "GSA", "GSAN"]

data = [gates, t_count, cliffords, cnot, other, hadamard, times]
dataframes.append(pd.DataFrame(data, columns=columns, index=rows))

INFO:root:Circuit  on 5 qubits with 56 gates.
        24 is the T-count
        32 Cliffords among which
        24 2-qubit gates (21 CNOT, 3 other) and
        6 Hadamard gates.


INFO:root:Loaded gf2^5_mult
INFO:root:Circuit  on 15 qubits with 323 gates.
        155 is the T-count
        168 Cliffords among which
        154 2-qubit gates (147 CNOT, 7 other) and
        14 Hadamard gates.
INFO:root:Loaded gf2^6_mult
INFO:root:Circuit  on 18 qubits with 465 gates.
        216 is the T-count
        249 Cliffords among which
        221 2-qubit gates (210 CNOT, 11 other) and
        22 Hadamard gates.
INFO:root:Loaded mod_red_21
INFO:root:Circuit  on 11 qubits with 245 gates.
        107 is the T-count
        138 Cliffords among which
        105 2-qubit gates (91 CNOT, 14 other) and
        30 Hadamard gates.


In [4]:
from overrides import override
from pyzx.circuit import Circuit
from pyzx.circuit.gates import CZ, Gate, ZPhase

from pyzx.optimize import Optimizer, toggle_element


def basic_optimization_min_cnots(circuit: Circuit, do_swaps:bool=True, quiet:bool=True) -> Circuit:
    """Optimizes the circuit using a strategy that involves delayed placement of gates
    so that more matches for gate cancellations are found. Specifically tries to minimize
    the number of Hadamard gates to improve the effectiveness 
    of phase-polynomial optimization techniques.

    Args:
        circuit: Circuit to be optimized.
        do_swaps: When set uses some rules transforming CNOT gates into SWAP gates. Generally leads to better results, but messes up architecture-aware placement of 2-qubit gates.
        quiet: Whether to print some progress indicators.
    """
    if not isinstance(circuit, Circuit):
        raise TypeError("Input must be a Circuit")
    o = Optimizer_no_new_cnots(circuit)
    return o.parse_circuit(do_swaps=do_swaps,quiet=quiet)

class Optimizer_no_new_cnots(Optimizer):
    """This class is a subclass of Optimizer that does not allow the creation of new CNOT gates."""

    def __init__(self, circuit: Circuit) -> None:
        super().__init__(circuit)

    @override
    def parse_gate(self, g: Gate) -> None:
        """The main function of the optimization. It records whether a gate needs to be placed at the specified location
        'right now', or whether we can postpone the placement until hopefully it is cancelled against some future gate.
        Only supports ZPhase, HAD, CNOT and CZ gates. """
        g = g.copy()
        # If we have some SWAPs recorded we need to change the target/control of the gate accordingly
        g.target = next(i for i in self.permutation if self.permutation[i] == g.target)
        t = g.target
        if g.name in ('CZ', 'CNOT'):
            g.control = next(i for i in self.permutation if self.permutation[i] == g.control)

        if g.name == 'HAD':
            # If we have recorded a NOT or Z gate at the target location, we push it trough the Hadamard and change the type
            if t in self.nots and t not in self.zs:
                self.nots.remove(t)
                self.zs.append(t)
            elif t in self.zs and t not in self.nots:
                self.zs.remove(t)
                self.nots.append(t)
            # See whether we have a HAD-S-HAD situation
            # And turn it into a S*-HAD-S* situation
            if len(self.gates[t])>1 and self.gates[t][-2].name == 'HAD' and isinstance(self.gates[t][-1], ZPhase):
                    g2 = self.gates[t][-1]
                    if g2.phase.denominator == 2:
                        h = self.gates[t][-2]
                        zp = ZPhase(t, (-g2.phase)%2)
                        zp.index = self.gcount
                        self.gcount += 1
                        g2.phase = zp.phase
                        if g2.name == 'S' and g2.phase.numerator > 1:
                            g2.adjoint = True
                        self.gates[t].insert(-2,zp)
                        return
            toggle_element(self.hadamards, t)
        elif g.name == 'NOT':
            toggle_element(self.nots, t)
        elif isinstance(g, ZPhase):
            if t in self.zs: #Consume a Z gate into the phase gate
                g.phase = (g.phase+1)%2
                self.zs.remove(t)
            if g.phase == 0: return
            if t in self.nots: # Push the phase gate trough a NOT
                g.phase = (-g.phase)%2
            if g.phase == 1: # If the resulting phase is a pi, then we record it as a Z gate
                toggle_element(self.zs, t)
                return
            if g.name == 'S':                           # We might have changed the phase, and therefore
                g.adjoint = g.phase.numerator != 1      # Need to adjust whether the adjoint is true
            if t in self.hadamards: # We can't push a phase gate trough a HAD, so we actually place the HAD down
                self.add_hadamard(t)
            if self.availty[t] == 1 and any(isinstance(g2, ZPhase) for g2 in self.available[t]): # There is an available phase gate
                i = next(i for i,g2 in enumerate(self.available[t]) if isinstance(g2, ZPhase))   # That we can fuse with the new one
                g2 = self.available[t].pop(i)
                self.gates[t].remove(g2)
                phase = (g.phase+g2.phase)%2
                if phase == 1:
                    toggle_element(self.zs, t)
                    return
                if phase != 0:
                    p = ZPhase(t, phase)
                    self.add_gate(t,p)
            else:
                if self.availty[t] == 2: # If previous gate was of X-type
                    self.availty[t] = 1  # We reset the available gates on this qubit
                    self.available[t] = list()
                g = ZPhase(t, g.phase)  # Avoid subclasses of ZPhase with inconsistent phase
                self.add_gate(t, g)
        elif g.name == 'CZ':
            t1, t2 = g.control, g.target
            if t1 > t2: # Normalise so that always g.target<g.control (since CZs are symmetric anyway)
                g.target = t1
                g.control = t2
            # Push NOT gates trough the CZ
            if t1 in self.nots: 
                toggle_element(self.zs, t2)
            if t2 in self.nots:
                toggle_element(self.zs, t1)
            # If there are HADs on both targets, we cannot commute the CZ trough and we place the HADs
            if t1 in self.hadamards and t2 in self.hadamards:
                self.add_hadamard(t1)
                self.add_hadamard(t2)
            if t1 not in self.hadamards and t2 not in self.hadamards:
                self.add_cz(g)
            # Exactly one of t1 and t2 has a hadamard
            # Do not allow the creation of new CNOT gates
            elif t1 in self.hadamards:
                self.add_hadamard(t1)
                self.add_cz(g)
            else:
                self.add_hadamard(t2)
                self.add_cz(g)
            
        elif g.name == 'CNOT':
            c, t = g.control, g.target
            # Commute NOTs and Zs trough the CNOT
            if c in self.nots:
                toggle_element(self.nots, t)
            if t in self.zs:
                toggle_element(self.zs, c)
            # If HADs are on both qubits, we commute the CNOT trough by switching target and control
            if c in self.hadamards and t in self.hadamards:
                g.control = t
                g.target = c
                self.add_cnot(g)
            elif c not in self.hadamards and t not in self.hadamards:
                self.add_cnot(g)
            # If there is a HAD on the target, the CNOT commutes trough to become a CZ
            elif t in self.hadamards:
                cz = CZ(c if c<t else t, c if c>t else t)
                self.add_cz(cz)
            else: # Only the control has a hadamard gate in front of it
                self.add_hadamard(c)
                self.add_cnot(g)
        
        else:
            raise TypeError("Unknown gate {}".format(str(g)))

In [5]:
def run_algorithm(algorithm, input_data, dataframes, pre_tr:bool = True):
    
    gates = []
    t_count = []
    cliffords = []
    cnot = []
    other = []
    hadamard = []
    times = []

    for name, circuit, graph in zip(input_data["Name"], input_data["circuit"], input_data["graph"]):
        graph_simplified = graph.clone()
        if pre_tr:
            graph_simplified = zx.simplify.teleport_reduce(graph_simplified)
            graph_simplified.track_phases = False

        logging.info(f"Running {algorithm} on {name}")

        start = time.perf_counter()
        algorithm(graph_simplified)
        end = time.perf_counter() - start

        logging.info(f"Finished execution in {end} seconds")

        qc = zx.extract_circuit(graph_simplified)
        try:
            # qc = basic_optimization_min_cnots(qc.to_basic_gates())
            qc = zx.optimize.basic_optimization(qc.to_basic_gates())
        except Exception as e:
            raise e

        stats = qc.stats()
        logging.info(stats)
        # Extract the numbers
        numbers = re.findall(r'\d+', stats)

        # Assign the numbers to variables
        gates.append(int(numbers[1]))
        t_count.append(int(numbers[2]))
        cliffords.append(int(numbers[3]))
        cnot.append(int(numbers[6]))
        other.append(int(numbers[7]))
        hadamard.append(int(numbers[8]))
        times.append(int(end))

    data = [gates, t_count, cliffords, cnot, other, hadamard, times]
    dataframes.append(pd.DataFrame(data, columns=columns, index=rows))

In [6]:
run_algorithm(zx.simplify.teleport_reduce, input_data, dataframes, pre_tr=False)

INFO:root:Running <function teleport_reduce at 0x0000022C36EEFCE0> on barenco_tof_3
INFO:root:Finished execution in 0.006451400000059948 seconds
INFO:root:Circuit  on 5 qubits with 116 gates.
        22 is the T-count
        94 Cliffords among which
        24 2-qubit gates (0 CNOT, 24 other) and
        66 Hadamard gates.
INFO:root:Running <function teleport_reduce at 0x0000022C36EEFCE0> on gf2^5_mult
INFO:root:Finished execution in 0.047008899999582354 seconds
INFO:root:Circuit  on 15 qubits with 753 gates.
        155 is the T-count
        598 Cliffords among which
        154 2-qubit gates (0 CNOT, 154 other) and
        444 Hadamard gates.
INFO:root:Running <function teleport_reduce at 0x0000022C36EEFCE0> on gf2^6_mult
INFO:root:Finished execution in 0.07088290000001507 seconds
INFO:root:Circuit  on 18 qubits with 1079 gates.
        216 is the T-count
        863 Cliffords among which
        221 2-qubit gates (0 CNOT, 221 other) and
        636 Hadamard gates.
INFO:root:Runnin

In [7]:
run_algorithm(zx.simplify.full_reduce, input_data, dataframes, pre_tr=False)

INFO:root:Running <function full_reduce at 0x0000022C36EEFC40> on barenco_tof_3
INFO:root:Finished execution in 0.009110399999826768 seconds
INFO:root:Circuit  on 5 qubits with 87 gates.
        16 is the T-count
        71 Cliffords among which
        40 2-qubit gates (24 CNOT, 16 other) and
        28 Hadamard gates.
INFO:root:Running <function full_reduce at 0x0000022C36EEFC40> on gf2^5_mult
INFO:root:Finished execution in 0.10128489999988233 seconds
INFO:root:Circuit  on 15 qubits with 815 gates.
        115 is the T-count
        700 Cliffords among which
        473 2-qubit gates (60 CNOT, 413 other) and
        225 Hadamard gates.
INFO:root:Running <function full_reduce at 0x0000022C36EEFC40> on gf2^6_mult
INFO:root:Finished execution in 0.2027760999999373 seconds
INFO:root:Circuit  on 18 qubits with 1307 gates.
        150 is the T-count
        1157 Cliffords among which
        806 2-qubit gates (97 CNOT, 709 other) and
        322 Hadamard gates.
INFO:root:Running <function

In [8]:
from functools import partial

for la in range(1):
    partial_la = partial(zx.simplify.greedy_simp, lookahead=la, threshold=0, use_neighbor_unfusion=True)
    run_algorithm(partial_la, input_data, dataframes)

INFO:root:Running functools.partial(<function greedy_simp at 0x0000022C36EEFD80>, lookahead=0, threshold=0, use_neighbor_unfusion=True) on barenco_tof_3
c:\Users\wsajk\Documents\Arbeit\MUNIQC-Atoms\pyzx-heuristics\demos\heuristic_demos\../..\pyzx\heuristics\neighbor_unfusion_simplification.py:692: UserWarning: Neighbor unfusion requires a flow function. Using G-flow preserving function.
  warnings.warn("Neighbor unfusion requires a flow function. Using G-flow preserving function.")
c:\Users\wsajk\Documents\Arbeit\MUNIQC-Atoms\pyzx-heuristics\demos\heuristic_demos\../..\pyzx\heuristics\neighbor_unfusion_simplification.py:1123: UserWarning: Std of reduction: 0.0 is too low. Stopping.
  warnings.warn(message=f"Std of reduction: {np.std(last_matches)} is too low. Stopping.")
INFO:root:Total rule applications: 42, Total reduction: 11.0, Std reduction: 0.6568149630539868
INFO:root:Total skipped filter function evaluations: 0, Total neighbor unfusions: 38, Total evaluations: 0, Total skipped 

In [ ]:
from functools import partial

for la in range(1):
    partial_la = partial(zx.simplify.greedy_simp, lookahead=la, threshold=0, use_neighbor_unfusion=False)
    run_algorithm(partial_la, input_data, dataframes)

In [ ]:
# run_algorithm(zx.simplify.sim_anneal_simp, input_data, dataframes)

In [ ]:
# run_algorithm(zx.simplify.sim_anneal_simp_neighbors, input_data, dataframes)

In [ ]:
df = pd.concat(dataframes, axis=0, keys=algorithm)
df.to_csv('benchmark_greedy_la.csv')